# Task 1: Klasifikasi

## Import Modules

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTENC

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.svm import SVC
from catboost import CatBoostClassifier

In [ ]:
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 200)

## Utilities

In [ ]:
from sklearn.metrics import precision_score, \
    recall_score, classification_report, \
    accuracy_score, f1_score


def evaluate_classifier_performance(prediction, y_test):
    # Informasi evaluasi secara compact
    print("Hasil Evaluasi berdasarkan classification report \n\n%s\n" % (classification_report(y_test, prediction,zero_division=0)))
    print()
    print("Confusion Matrix")
    print()
    y_actual = pd.Series(np.array(y_test), name = "actual")
    y_pred = pd.Series(np.array(prediction), name = "prediction")
    df_confusion = pd.crosstab(y_actual, y_pred)
    display(df_confusion)
    print()
    print()

    print("Butuh informasi lebih lengkap? silakan simak di bawah ini : ")
    print('Accuracy Average:', accuracy_score(y_test, prediction))
    print('F1 Macro Average:', f1_score(y_test, prediction, average='macro'))
    print('F1 Micro Average:', f1_score(y_test, prediction, average='micro'))
    print('Precision Macro Average:', precision_score(y_test, prediction, average='macro',zero_division=0))
    print('Precision Micro Average:', precision_score(y_test, prediction, average='micro',zero_division=0))
    print('Recall Macro Average:', recall_score(y_test, prediction, average='macro',zero_division=0))
    print('Recall Micro Average:', recall_score(y_test, prediction, average='micro',zero_division=0))
    print()

## Load data

In [ ]:
ufc = pd.read_csv("./UFC_train.csv")
ufc

,R_fighter,B_fighter,Referee,date,location,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,B_avg_opp_REV,B_avg_SIG_STR_att,B_avg_SIG_STR_landed,B_avg_opp_SIG_STR_att,B_avg_opp_SIG_STR_landed,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_avg_opp_TOTAL_STR_att,B_avg_opp_TOTAL_STR_landed,B_avg_TD_att,B_avg_TD_landed,B_avg_opp_TD_att,B_avg_opp_TD_landed,B_avg_HEAD_att,B_avg_HEAD_landed,B_avg_opp_HEAD_att,B_avg_opp_HEAD_landed,B_avg_BODY_att,B_avg_BODY_landed,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,B_avg_LEG_att,B_avg_LEG_landed,B_avg_opp_LEG_att,B_avg_opp_LEG_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,B_avg_opp_DISTANCE_att,B_avg_opp_DISTANCE_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_opp_CLINCH_att,B_avg_opp_CLINCH_landed,B_avg_GROUND_att,B_avg_GROUND_landed,B_avg_opp_GROUND_att,B_avg_opp_GROUND_landed,B_avg_CTRL_time(seconds),B_avg_opp_CTRL_time(seconds),B_total_time_fought(seconds),B_total_rounds_fought,B_total_title_bouts,B_current_win_streak,B_current_lose_streak,B_longest_win_streak,B_wins,B_losses,B_draw,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,R_avg_KD,R_avg_opp_KD,R_avg_SIG_STR_pct,R_avg_opp_SIG_STR_pct,R_avg_TD_pct,R_avg_opp_TD_pct,R_avg_SUB_ATT,R_avg_opp_SUB_ATT,R_avg_REV,R_avg_opp_REV,R_avg_SIG_STR_att,R_avg_SIG_STR_landed,R_avg_opp_SIG_STR_att,R_avg_opp_SIG_STR_landed,R_avg_TOTAL_STR_att,R_avg_TOTAL_STR_landed,R_avg_opp_TOTAL_STR_att,R_avg_opp_TOTAL_STR_landed,R_avg_TD_att,R_avg_TD_landed,R_avg_opp_TD_att,R_avg_opp_TD_landed,R_avg_HEAD_att,R_avg_HEAD_landed,R_avg_opp_HEAD_att,R_avg_opp_HEAD_landed,R_avg_BODY_att,R_avg_BODY_landed,R_avg_opp_BODY_att,R_avg_opp_BODY_landed,R_avg_LEG_att,R_avg_LEG_landed,R_avg_opp_LEG_att,R_avg_opp_LEG_landed,R_avg_DISTANCE_att,R_avg_DISTANCE_landed,R_avg_opp_DISTANCE_att,R_avg_opp_DISTANCE_landed,R_avg_CLINCH_att,R_avg_CLINCH_landed,R_avg_opp_CLINCH_att,R_avg_opp_CLINCH_landed,R_avg_GROUND_att,R_avg_GROUND_landed,R_avg_opp_GROUND_att,R_avg_opp_GROUND_landed,R_avg_CTRL_time(seconds),R_avg_opp_CTRL_time(seconds),R_total_time_fought(seconds),R_total_rounds_fought,R_total_title_bouts,R_current_win_streak,R_current_lose_streak,R_longest_win_streak,R_wins,R_losses,R_draw,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,Winner
0,Joe Riggs,Joe Doerksen,Steve Mazzagatti,2004-08-21,"Las Vegas, Nevada, USA",False,Middleweight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Orthodox,182.88,190.50,185.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Southpaw,182.88,177.80,185.0,26.0,21.0,Red
1,Jorge Masvidal,Al Iaquinta,Keith Peterson,2015-04-04,"Fairfax, Virginia, USA",False,Lightweight,1.156250,0.000000,0.394141,0.352422,0.239219,0.011484,0.156250,0.132812,0.000000,0.000000,137.460938,53.234375,89.953125,31.296875,141.210938,56.718750,91.882812,33.078125,2.000000,0.625000,1.835938,0.085938,117.312500,39.976562,83.210938,27.132812,13.625000,7.765625,3.960938,2.070312,6.523438,5.492188,2.781250,2.093750,127.031250,46.812500,88.695312,30.882812,5.671875,3.679688,0.710938,0.054688,4.757812,2.742188,0.546875,0.359375,35.945312,14.601562,522.835938,19,1,0,1,3,6,2,0,0,0,3,3,0,0,Orthodox,177.80,177.80,155.0,0.500000,0.343750,0.498750,0.339062,0.554688,0.207500,0.562500,0.312500,0.500000,0.031250,124.500000,60.562500,117.812500,41.843750,156.0

In [ ]:
ufc.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5410 entries, 0 to 5409
Data columns (total 144 columns):
 #    Column                        Dtype  
---   ------                        -----  
 0    R_fighter                     object 
 1    B_fighter                     object 
 2    Referee                       object 
 3    date                          object 
 4    location                      object 
 5    title_bout                    bool   
 6    weight_class                  object 
 7    B_avg_KD                      float64
 8    B_avg_opp_KD                  float64
 9    B_avg_SIG_STR_pct             float64
 10   B_avg_opp_SIG_STR_pct         float64
 11   B_avg_TD_pct                  float64
 12   B_avg_opp_TD_pct              float64
 13   B_avg_SUB_ATT                 float64
 14   B_avg_opp_SUB_ATT             float64
 15   B_avg_REV                     float64
 16   B_avg_opp_REV                 float64
 17   B_avg_SIG_STR_att             float64
 18   B_avg_

In [ ]:
X_train, y_train = ufc.drop(axis=1, columns=["Winner"]), ufc[["Winner"]]

In [ ]:
X_train.head()

,R_fighter,B_fighter,Referee,date,location,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,B_avg_opp_REV,B_avg_SIG_STR_att,B_avg_SIG_STR_landed,B_avg_opp_SIG_STR_att,B_avg_opp_SIG_STR_landed,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_avg_opp_TOTAL_STR_att,B_avg_opp_TOTAL_STR_landed,B_avg_TD_att,B_avg_TD_landed,B_avg_opp_TD_att,B_avg_opp_TD_landed,B_avg_HEAD_att,B_avg_HEAD_landed,B_avg_opp_HEAD_att,B_avg_opp_HEAD_landed,B_avg_BODY_att,B_avg_BODY_landed,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,B_avg_LEG_att,B_avg_LEG_landed,B_avg_opp_LEG_att,B_avg_opp_LEG_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,B_avg_opp_DISTANCE_att,B_avg_opp_DISTANCE_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_opp_CLINCH_att,B_avg_opp_CLINCH_landed,B_avg_GROUND_att,B_avg_GROUND_landed,B_avg_opp_GROUND_att,B_avg_opp_GROUND_landed,B_avg_CTRL_time(seconds),B_avg_opp_CTRL_time(seconds),B_total_time_fought(seconds),B_total_rounds_fought,B_total_title_bouts,B_current_win_streak,B_current_lose_streak,B_longest_win_streak,B_wins,B_losses,B_draw,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,R_avg_KD,R_avg_opp_KD,R_avg_SIG_STR_pct,R_avg_opp_SIG_STR_pct,R_avg_TD_pct,R_avg_opp_TD_pct,R_avg_SUB_ATT,R_avg_opp_SUB_ATT,R_avg_REV,R_avg_opp_REV,R_avg_SIG_STR_att,R_avg_SIG_STR_landed,R_avg_opp_SIG_STR_att,R_avg_opp_SIG_STR_landed,R_avg_TOTAL_STR_att,R_avg_TOTAL_STR_landed,R_avg_opp_TOTAL_STR_att,R_avg_opp_TOTAL_STR_landed,R_avg_TD_att,R_avg_TD_landed,R_avg_opp_TD_att,R_avg_opp_TD_landed,R_avg_HEAD_att,R_avg_HEAD_landed,R_avg_opp_HEAD_att,R_avg_opp_HEAD_landed,R_avg_BODY_att,R_avg_BODY_landed,R_avg_opp_BODY_att,R_avg_opp_BODY_landed,R_avg_LEG_att,R_avg_LEG_landed,R_avg_opp_LEG_att,R_avg_opp_LEG_landed,R_avg_DISTANCE_att,R_avg_DISTANCE_landed,R_avg_opp_DISTANCE_att,R_avg_opp_DISTANCE_landed,R_avg_CLINCH_att,R_avg_CLINCH_landed,R_avg_opp_CLINCH_att,R_avg_opp_CLINCH_landed,R_avg_GROUND_att,R_avg_GROUND_landed,R_avg_opp_GROUND_att,R_avg_opp_GROUND_landed,R_avg_CTRL_time(seconds),R_avg_opp_CTRL_time(seconds),R_total_time_fought(seconds),R_total_rounds_fought,R_total_title_bouts,R_current_win_streak,R_current_lose_streak,R_longest_win_streak,R_wins,R_losses,R_draw,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Joe Riggs,Joe Doerksen,Steve Mazzagatti,2004-08-21,"Las Vegas, Nevada, USA",False,Middleweight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Orthodox,182.88,190.50,185.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Southpaw,182.88,177.80,185.0,26.0,21.0
1,Jorge Masvidal,Al Iaquinta,Keith Peterson,2015-04-04,"Fairfax, Virginia, USA",False,Lightweight,1.156250,0.000000,0.394141,0.352422,0.239219,0.011484,0.15625,0.132812,0.000,0.0,137.460938,53.234375,89.953125,31.296875,141.210938,56.718750,91.882812,33.078125,2.000000,0.625000,1.835938,0.085938,117.312500,39.976562,83.210938,27.132812,13.625000,7.765625,3.960938,2.070312,6.523438,5.492188,2.781250,2.093750,127.031250,46.812500,88.695312,30.882812,5.671875,3.679688,0.710938,0.054688,4.757812,2.742188,0.546875,0.359375,35.945312,14.601562,522.835938,19,1,0,1,3,6,2,0,0,0,3,3,0,0,Orthodox,177.80,177.80,155.0,0.500000,0.343750,0.498750,0.339062,0.554688,0.207500,0.562500,0.312500,0.500000,0.031250,124.500000,60.562500,117.812500,41.843750,156.031250,87.593750,129.

In [ ]:
y_train.head()

,Winner
0,Red
1,Blue
2,Blue
3,Blue
4,Red


## Preprocessing

### Handling numerical values

In [ ]:
ufc.info(verbose=True, max_cols=200)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5410 entries, 0 to 5409
Data columns (total 144 columns):
 #    Column                        Non-Null Count  Dtype  
---   ------                        --------------  -----  
 0    R_fighter                     5410 non-null   object 
 1    B_fighter                     5410 non-null   object 
 2    Referee                       5379 non-null   object 
 3    date                          5410 non-null   object 
 4    location                      5410 non-null   object 
 5    title_bout                    5410 non-null   bool   
 6    weight_class                  5410 non-null   object 
 7    B_avg_KD                      4117 non-null   float64
 8    B_avg_opp_KD                  4117 non-null   float64
 9    B_avg_SIG_STR_pct             4117 non-null   float64
 10   B_avg_opp_SIG_STR_pct         4117 non-null   float64
 11   B_avg_TD_pct                  4117 non-null   float64
 12   B_avg_opp_TD_pct              4117 non-null   

In [ ]:
y_train.value_counts()

Winner
Red       3581
Blue      1730
Draw        99
Name: count, dtype: int64

In [ ]:
X_train.isnull().sum()

R_fighter                          0
B_fighter                          0
Referee                           31
date                               0
location                           0
title_bout                         0
weight_class                       0
B_avg_KD                        1293
B_avg_opp_KD                    1293
B_avg_SIG_STR_pct               1293
B_avg_opp_SIG_STR_pct           1293
B_avg_TD_pct                    1293
B_avg_opp_TD_pct                1293
B_avg_SUB_ATT                   1293
B_avg_opp_SUB_ATT               1293
B_avg_REV                       1293
B_avg_opp_REV                   1293
B_avg_SIG_STR_att               1293
B_avg_SIG_STR_landed            1293
B_avg_opp_SIG_STR_att           1293
B_avg_opp_SIG_STR_landed        1293
B_avg_TOTAL_STR_att             1293
B_avg_TOTAL_STR_landed          1293
B_avg_opp_TOTAL_STR_att         1293
B_avg_opp_TOTAL_STR_landed      1293
B_avg_TD_att                    1293
B_avg_TD_landed                 1293
B

In [ ]:
ufc_num = ufc.select_dtypes(exclude=["object", "bool"])
ufc_num.head()

,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,B_avg_opp_REV,B_avg_SIG_STR_att,B_avg_SIG_STR_landed,B_avg_opp_SIG_STR_att,B_avg_opp_SIG_STR_landed,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_avg_opp_TOTAL_STR_att,B_avg_opp_TOTAL_STR_landed,B_avg_TD_att,B_avg_TD_landed,B_avg_opp_TD_att,B_avg_opp_TD_landed,B_avg_HEAD_att,B_avg_HEAD_landed,B_avg_opp_HEAD_att,B_avg_opp_HEAD_landed,B_avg_BODY_att,B_avg_BODY_landed,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,B_avg_LEG_att,B_avg_LEG_landed,B_avg_opp_LEG_att,B_avg_opp_LEG_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,B_avg_opp_DISTANCE_att,B_avg_opp_DISTANCE_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_opp_CLINCH_att,B_avg_opp_CLINCH_landed,B_avg_GROUND_att,B_avg_GROUND_landed,B_avg_opp_GROUND_att,B_avg_opp_GROUND_landed,B_avg_CTRL_time(seconds),B_avg_opp_CTRL_time(seconds),B_total_time_fought(seconds),B_total_rounds_fought,B_total_title_bouts,B_current_win_streak,B_current_lose_streak,B_longest_win_streak,B_wins,B_losses,B_draw,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_Height_cms,B_Reach_cms,B_Weight_lbs,R_avg_KD,R_avg_opp_KD,R_avg_SIG_STR_pct,R_avg_opp_SIG_STR_pct,R_avg_TD_pct,R_avg_opp_TD_pct,R_avg_SUB_ATT,R_avg_opp_SUB_ATT,R_avg_REV,R_avg_opp_REV,R_avg_SIG_STR_att,R_avg_SIG_STR_landed,R_avg_opp_SIG_STR_att,R_avg_opp_SIG_STR_landed,R_avg_TOTAL_STR_att,R_avg_TOTAL_STR_landed,R_avg_opp_TOTAL_STR_att,R_avg_opp_TOTAL_STR_landed,R_avg_TD_att,R_avg_TD_landed,R_avg_opp_TD_att,R_avg_opp_TD_landed,R_avg_HEAD_att,R_avg_HEAD_landed,R_avg_opp_HEAD_att,R_avg_opp_HEAD_landed,R_avg_BODY_att,R_avg_BODY_landed,R_avg_opp_BODY_att,R_avg_opp_BODY_landed,R_avg_LEG_att,R_avg_LEG_landed,R_avg_opp_LEG_att,R_avg_opp_LEG_landed,R_avg_DISTANCE_att,R_avg_DISTANCE_landed,R_avg_opp_DISTANCE_att,R_avg_opp_DISTANCE_landed,R_avg_CLINCH_att,R_avg_CLINCH_landed,R_avg_opp_CLINCH_att,R_avg_opp_CLINCH_landed,R_avg_GROUND_att,R_avg_GROUND_landed,R_avg_opp_GROUND_att,R_avg_opp_GROUND_landed,R_avg_CTRL_time(seconds),R_avg_opp_CTRL_time(seconds),R_total_time_fought(seconds),R_total_rounds_fought,R_total_title_bouts,R_current_win_streak,R_current_lose_streak,R_longest_win_streak,R_wins,R_losses,R_draw,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,182.88,190.50,185.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,182.88,177.80,185.0,26.0,21.0
1,1.156250,0.000000,0.394141,0.352422,0.239219,0.011484,0.15625,0.132812,0.000,0.0,137.460938,53.234375,89.953125,31.296875,141.210938,56.718750,91.882812,33.078125,2.000000,0.625000,1.835938,0.085938,117.312500,39.976562,83.210938,27.132812,13.625000,7.765625,3.960938,2.070312,6.523438,5.492188,2.781250,2.093750,127.031250,46.812500,88.695312,30.882812,5.671875,3.679688,0.710938,0.054688,4.757812,2.742188,0.546875,0.359375,35.945312,14.601562,522.835938,19,1,0,1,3,6,2,0,0,0,3,3,0,0,177.80,177.80,155.0,0.500000,0.343750,0.498750,0.339062,0.554688,0.207500,0.562500,0.312500,0.500000,0.031250,124.500000,60.562500,117.812500,41.843750,156.031250,87.593750,129.593750,53.093750,3.500000,1.875000,5.812500,0.937500,93.468750,37.781250,92.250000,27.656250,21.656250,17.468750,17.593750,8.312500,9.375000,5.312500,7.968750,5.875000,98.250000,41.562500,110.250000,38.406250,14.281250,11.750000,4.562500,2.343750,11.968750,7.250000,3.000000,1.093750,252.312500,115.000

In [ ]:
ufc_num.isnull().sum()

B_avg_KD                        1293
B_avg_opp_KD                    1293
B_avg_SIG_STR_pct               1293
B_avg_opp_SIG_STR_pct           1293
B_avg_TD_pct                    1293
B_avg_opp_TD_pct                1293
B_avg_SUB_ATT                   1293
B_avg_opp_SUB_ATT               1293
B_avg_REV                       1293
B_avg_opp_REV                   1293
B_avg_SIG_STR_att               1293
B_avg_SIG_STR_landed            1293
B_avg_opp_SIG_STR_att           1293
B_avg_opp_SIG_STR_landed        1293
B_avg_TOTAL_STR_att             1293
B_avg_TOTAL_STR_landed          1293
B_avg_opp_TOTAL_STR_att         1293
B_avg_opp_TOTAL_STR_landed      1293
B_avg_TD_att                    1293
B_avg_TD_landed                 1293
B_avg_opp_TD_att                1293
B_avg_opp_TD_landed             1293
B_avg_HEAD_att                  1293
B_avg_HEAD_landed               1293
B_avg_opp_HEAD_att              1293
B_avg_opp_HEAD_landed           1293
B_avg_BODY_att                  1293
B

### Dropping columns not in test set

It seems there are no `B_Reach_cms` column in the test set we'll also remove it here.

In [ ]:
ufc_num = ufc_num.drop(axis=1, columns=["B_Reach_cms"], errors="ignore")

### KNN Imputation

Since a majority of these columns have missing values which are pretty significant in percentage, it's probably not missing at random. Between median and KNN, KNN imputation gave the better results so we will use that.

In [ ]:
num_imputer = KNNImputer()
ufc_num = pd.DataFrame(num_imputer.fit_transform(ufc_num), columns=ufc_num.columns)
ufc_num.isnull().sum()

B_avg_KD                        0
B_avg_opp_KD                    0
B_avg_SIG_STR_pct               0
B_avg_opp_SIG_STR_pct           0
B_avg_TD_pct                    0
B_avg_opp_TD_pct                0
B_avg_SUB_ATT                   0
B_avg_opp_SUB_ATT               0
B_avg_REV                       0
B_avg_opp_REV                   0
B_avg_SIG_STR_att               0
B_avg_SIG_STR_landed            0
B_avg_opp_SIG_STR_att           0
B_avg_opp_SIG_STR_landed        0
B_avg_TOTAL_STR_att             0
B_avg_TOTAL_STR_landed          0
B_avg_opp_TOTAL_STR_att         0
B_avg_opp_TOTAL_STR_landed      0
B_avg_TD_att                    0
B_avg_TD_landed                 0
B_avg_opp_TD_att                0
B_avg_opp_TD_landed             0
B_avg_HEAD_att                  0
B_avg_HEAD_landed               0
B_avg_opp_HEAD_att              0
B_avg_opp_HEAD_landed           0
B_avg_BODY_att                  0
B_avg_BODY_landed               0
B_avg_opp_BODY_att              0
B_avg_opp_BODY

### Handling categorical values

### Dropping not-so-useful features

In [ ]:
ufc_cat = ufc.select_dtypes(include=["object", "bool"]).drop(axis=1, columns=["Winner"], errors="ignore")
ufc_cat.columns

Index(['R_fighter', 'B_fighter', 'Referee', 'date', 'location', 'title_bout',
       'weight_class', 'B_Stance', 'R_Stance'],
      dtype='object')

Ideally, because R_fighter and B_fighter are names, we should just remove those features

In [ ]:
ufc_cat = ufc_cat.drop(axis=1, columns=["R_fighter", "B_fighter"], errors="ignore")
ufc_cat.columns

Index(['Referee', 'date', 'location', 'title_bout', 'weight_class', 'B_Stance',
       'R_Stance'],
      dtype='object')

Unique values for each columns

In [ ]:
ufc_cat["location"].value_counts()

location
Las Vegas, Nevada, USA                         1416
Abu Dhabi, Abu Dhabi, United Arab Emirates      135
London, England, United Kingdom                 101
Newark, New Jersey, USA                          80
Montreal, Quebec, Canada                         77
Anaheim, California, USA                         76
Atlantic City, New Jersey, USA                   73
Chicago, Illinois, USA                           71
Los Angeles, California, USA                     69
Stockholm, Sweden                                67
Rio de Janeiro, Brazil                           67
Toronto, Ontario, Canada                         65
Boston, Massachusetts, USA                       64
Houston, Texas, USA                              62
Denver, Colorado, USA                            58
Sacramento, California, USA                      58
Dallas, Texas, USA                               57
Sao Paulo, Sao Paulo, Brazil                     56
Nashville, Tennessee, USA                        55
Mex

Since there are so many locations and we don't know the nationality of these fighters, most likely it won't help the models. Dates we can ignore for now too

In [ ]:
ufc_cat = ufc_cat.drop(axis=1, columns=["location", "date"], errors="ignore")

In [ ]:
ufc_cat.columns

Index(['Referee', 'title_bout', 'weight_class', 'B_Stance', 'R_Stance'], dtype='object')

In [ ]:
ufc_cat["Referee"].value_counts()

Referee
Herb Dean               780
John McCarthy           585
Dan Miragliotta         358
Mario Yamasaki          352
Marc Goddard            311
                       ... 
Brandon Pfannenstiel      1
Mark Lawley               1
Marcio Laselva            1
Steven Davis              1
Robbie Scheureumann       1
Name: count, Length: 202, dtype: int64

Referee can be removed for the same reasons as described

In [ ]:
ufc_cat = ufc_cat.drop(axis=1, columns=["Referee"], errors="ignore")

In [ ]:
ufc_cat.columns

Index(['title_bout', 'weight_class', 'B_Stance', 'R_Stance'], dtype='object')

In [ ]:
ufc_cat["title_bout"].value_counts()

title_bout
False    5080
True      330
Name: count, dtype: int64

In [ ]:
ufc_cat["weight_class"].value_counts()

weight_class
Welterweight          980
Lightweight           967
Middleweight          734
Heavyweight           538
LightHeavyweight      511
Featherweight         490
Bantamweight          427
Flyweight             208
WomenStrawweight      176
WomenBantamweight     132
WomenFlyweight        104
OpenWeight             77
CatchWeight            50
WomenFeatherweight     16
Name: count, dtype: int64

In [ ]:
ufc_cat["B_Stance"].value_counts()

B_Stance
Orthodox       4077
Southpaw       1048
Switch          209
Open Stance       8
Sideways          4
Name: count, dtype: int64

In [ ]:
ufc_cat["R_Stance"].value_counts()

R_Stance
Orthodox       4077
Southpaw       1110
Switch          180
Open Stance      14
Sideways          2
Name: count, dtype: int64

These features might be useful. There's also a heavy imbalance in the features but this might be due to the nature of it since some fighting styles are more common than others, so we'll ignore that

Let's come back to this later

### Encoding the categorical variables using One-Hot encoding

Since we ended up using CatBoost, we didn't really need to encode it directly and let CatBoost do the job with its own version of one-hot encoding. But this is a sample of what we basically did.

In [ ]:
cat_enc = OneHotEncoder()

ufc_cat = cat_enc.fit_transform(ufc_cat).toarray()

In [ ]:
ufc_cat = pd.DataFrame(ufc_cat, columns=cat_enc.get_feature_names_out())
ufc_cat.head()

,title_bout_False,title_bout_True,weight_class_Bantamweight,weight_class_CatchWeight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,weight_class_LightHeavyweight,weight_class_Lightweight,weight_class_Middleweight,weight_class_OpenWeight,weight_class_Welterweight,weight_class_WomenBantamweight,weight_class_WomenFeatherweight,weight_class_WomenFlyweight,weight_class_WomenStrawweight,B_Stance_Open Stance,B_Stance_Orthodox,B_Stance_Sideways,B_Stance_Southpaw,B_Stance_Switch,B_Stance_nan,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Sideways,R_Stance_Southpaw,R_Stance_Switch,R_Stance_nan
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
ufc_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5410 entries, 0 to 5409
Data columns (total 28 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   title_bout_False                 5410 non-null   float64
 1   title_bout_True                  5410 non-null   float64
 2   weight_class_Bantamweight        5410 non-null   float64
 3   weight_class_CatchWeight         5410 non-null   float64
 4   weight_class_Featherweight       5410 non-null   float64
 5   weight_class_Flyweight           5410 non-null   float64
 6   weight_class_Heavyweight         5410 non-null   float64
 7   weight_class_LightHeavyweight    5410 non-null   float64
 8   weight_class_Lightweight         5410 non-null   float64
 9   weight_class_Middleweight        5410 non-null   float64
 10  weight_class_OpenWeight          5410 non-null   float64
 11  weight_class_Welterweight        5410 non-null   float64
 12  weight_class_WomenBa

In [ ]:
ufc_cat[ufc_cat.select_dtypes(include=["bool"]).columns] = ufc_cat.select_dtypes(include=["bool"]).replace({True: "True", False: "False"})

### Mode Imputation

In [ ]:
ufc_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5410 entries, 0 to 5409
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title_bout    5410 non-null   object
 1   weight_class  5410 non-null   object
 2   B_Stance      5346 non-null   object
 3   R_Stance      5383 non-null   object
dtypes: object(4)
memory usage: 169.2+ KB


This time, let's impute with mode.

In [ ]:
cat_imputer = SimpleImputer(strategy="most_frequent")
ufc_cat = pd.DataFrame(cat_imputer.fit_transform(ufc_cat), columns=ufc_cat.columns)
ufc_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5410 entries, 0 to 5409
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title_bout    5410 non-null   object
 1   weight_class  5410 non-null   object
 2   B_Stance      5410 non-null   object
 3   R_Stance      5410 non-null   object
dtypes: object(4)
memory usage: 169.2+ KB


## Splitting between features and target

In [ ]:
ufc_prepped = ufc_num.copy()
ufc_prepped[ufc_cat.columns] = ufc_cat

In [ ]:
ufc_prepped

,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,B_avg_opp_REV,B_avg_SIG_STR_att,B_avg_SIG_STR_landed,B_avg_opp_SIG_STR_att,B_avg_opp_SIG_STR_landed,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_avg_opp_TOTAL_STR_att,B_avg_opp_TOTAL_STR_landed,B_avg_TD_att,B_avg_TD_landed,B_avg_opp_TD_att,B_avg_opp_TD_landed,B_avg_HEAD_att,B_avg_HEAD_landed,B_avg_opp_HEAD_att,B_avg_opp_HEAD_landed,B_avg_BODY_att,B_avg_BODY_landed,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,B_avg_LEG_att,B_avg_LEG_landed,B_avg_opp_LEG_att,B_avg_opp_LEG_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,B_avg_opp_DISTANCE_att,B_avg_opp_DISTANCE_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_opp_CLINCH_att,B_avg_opp_CLINCH_landed,B_avg_GROUND_att,B_avg_GROUND_landed,B_avg_opp_GROUND_att,B_avg_opp_GROUND_landed,B_avg_CTRL_time(seconds),B_avg_opp_CTRL_time(seconds),B_total_time_fought(seconds),B_total_rounds_fought,B_total_title_bouts,B_current_win_streak,B_current_lose_streak,B_longest_win_streak,B_wins,B_losses,B_draw,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_Height_cms,B_Weight_lbs,R_avg_KD,R_avg_opp_KD,R_avg_SIG_STR_pct,R_avg_opp_SIG_STR_pct,R_avg_TD_pct,R_avg_opp_TD_pct,R_avg_SUB_ATT,R_avg_opp_SUB_ATT,R_avg_REV,R_avg_opp_REV,R_avg_SIG_STR_att,R_avg_SIG_STR_landed,R_avg_opp_SIG_STR_att,R_avg_opp_SIG_STR_landed,R_avg_TOTAL_STR_att,R_avg_TOTAL_STR_landed,R_avg_opp_TOTAL_STR_att,R_avg_opp_TOTAL_STR_landed,R_avg_TD_att,R_avg_TD_landed,R_avg_opp_TD_att,R_avg_opp_TD_landed,R_avg_HEAD_att,R_avg_HEAD_landed,R_avg_opp_HEAD_att,R_avg_opp_HEAD_landed,R_avg_BODY_att,R_avg_BODY_landed,R_avg_opp_BODY_att,R_avg_opp_BODY_landed,R_avg_LEG_att,R_avg_LEG_landed,R_avg_opp_LEG_att,R_avg_opp_LEG_landed,R_avg_DISTANCE_att,R_avg_DISTANCE_landed,R_avg_opp_DISTANCE_att,R_avg_opp_DISTANCE_landed,R_avg_CLINCH_att,R_avg_CLINCH_landed,R_avg_opp_CLINCH_att,R_avg_opp_CLINCH_landed,R_avg_GROUND_att,R_avg_GROUND_landed,R_avg_opp_GROUND_att,R_avg_opp_GROUND_landed,R_avg_CTRL_time(seconds),R_avg_opp_CTRL_time(seconds),R_total_time_fought(seconds),R_total_rounds_fought,R_total_title_bouts,R_current_win_streak,R_current_lose_streak,R_longest_win_streak,R_wins,R_losses,R_draw,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,title_bout,weight_class,B_Stance,R_Stance
0,0.400000,0.200000,0.387000,0.354000,0.194000,0.160000,0.000000,1.200000,0.000000,0.000000,71.300000,32.250000,40.200000,14.000000,101.950000,57.950000,59.750000,32.000000,2.650000,1.050000,1.650000,0.550000,58.500000,24.000000,31.950000,8.100000,7.850000,4.100000,3.650000,1.950000,4.950000,4.150000,4.600000,3.950000,45.500000,16.900000,30.000000,7.750000,5.200000,4.100000,2.550000,1.900000,20.600000,11.250000,7.650000,4.350000,156.150000,98.100000,538.300000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,182.88,185.0,0.500000,0.200000,0.599000,0.413000,0.800000,0.232000,0.800000,0.600000,0.000000,0.200000,19.100000,10.600000,21.200000,12.300000,24.500000,15.400000,49.100000,38.400000,1.100000,1.100000,1.500000,0.900000,15.800000,7.800000,18.600000,10.500000,2.600000,2.400000,1.800000,1.200000,0.700000,0.400000,0.800000,0.600000,9.000000,4.000000,7.000000,1.600000,3.100000,2.300000,3.500000,2.900000,7.000000,4.300000,10.700000,7.800000,95.500000,176.500000,327.900000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,182.88,177.80,185.0,26.0,21.0,False,Middleweight,Orthodox,Southpaw
1,1.156250,0.000000,0.394141,0.352422,0.239219,0.011484,0.156250,0.132812,0.000000,0.000000,137.460938,53.234375,89.953125,31.296875,141.210938,56.718750,91.882812,33.078125,2.000000,0.625000,1.835938,0.085938,117.312500,39.976562,83.210938,27.132812,13.625000,7.765625,3.960938,2.070312,6.523438,5.492188,2.781250,2.09375

In [ ]:
X, y = ufc_prepped, ufc[["Winner"]]

### Oversampling using SMOTE-NC

Since the features are all either nominal or continuous/numerical and that we have a pretty severe imbalance, we opted for a variation on SMOTE for oversampling which can handle nominal categorical and numeric data.

In [ ]:
y.value_counts()

Winner
Red       3581
Blue      1730
Draw        99
Name: count, dtype: int64

In [ ]:
smote = SMOTENC(categorical_features=ufc_cat.columns.tolist())
X_smote, y_smote = smote.fit_resample(X, y)

In [ ]:
y_smote.value_counts()

Winner
Blue      3581
Draw      3581
Red       3581
Name: count, dtype: int64

### Train test split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42, stratify=y_smote)

In [ ]:
X_train

,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,B_avg_opp_REV,B_avg_SIG_STR_att,B_avg_SIG_STR_landed,B_avg_opp_SIG_STR_att,B_avg_opp_SIG_STR_landed,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_avg_opp_TOTAL_STR_att,B_avg_opp_TOTAL_STR_landed,B_avg_TD_att,B_avg_TD_landed,B_avg_opp_TD_att,B_avg_opp_TD_landed,B_avg_HEAD_att,B_avg_HEAD_landed,B_avg_opp_HEAD_att,B_avg_opp_HEAD_landed,B_avg_BODY_att,B_avg_BODY_landed,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,B_avg_LEG_att,B_avg_LEG_landed,B_avg_opp_LEG_att,B_avg_opp_LEG_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,B_avg_opp_DISTANCE_att,B_avg_opp_DISTANCE_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_opp_CLINCH_att,B_avg_opp_CLINCH_landed,B_avg_GROUND_att,B_avg_GROUND_landed,B_avg_opp_GROUND_att,B_avg_opp_GROUND_landed,B_avg_CTRL_time(seconds),B_avg_opp_CTRL_time(seconds),B_total_time_fought(seconds),B_total_rounds_fought,B_total_title_bouts,B_current_win_streak,B_current_lose_streak,B_longest_win_streak,B_wins,B_losses,B_draw,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_Height_cms,B_Weight_lbs,R_avg_KD,R_avg_opp_KD,R_avg_SIG_STR_pct,R_avg_opp_SIG_STR_pct,R_avg_TD_pct,R_avg_opp_TD_pct,R_avg_SUB_ATT,R_avg_opp_SUB_ATT,R_avg_REV,R_avg_opp_REV,R_avg_SIG_STR_att,R_avg_SIG_STR_landed,R_avg_opp_SIG_STR_att,R_avg_opp_SIG_STR_landed,R_avg_TOTAL_STR_att,R_avg_TOTAL_STR_landed,R_avg_opp_TOTAL_STR_att,R_avg_opp_TOTAL_STR_landed,R_avg_TD_att,R_avg_TD_landed,R_avg_opp_TD_att,R_avg_opp_TD_landed,R_avg_HEAD_att,R_avg_HEAD_landed,R_avg_opp_HEAD_att,R_avg_opp_HEAD_landed,R_avg_BODY_att,R_avg_BODY_landed,R_avg_opp_BODY_att,R_avg_opp_BODY_landed,R_avg_LEG_att,R_avg_LEG_landed,R_avg_opp_LEG_att,R_avg_opp_LEG_landed,R_avg_DISTANCE_att,R_avg_DISTANCE_landed,R_avg_opp_DISTANCE_att,R_avg_opp_DISTANCE_landed,R_avg_CLINCH_att,R_avg_CLINCH_landed,R_avg_opp_CLINCH_att,R_avg_opp_CLINCH_landed,R_avg_GROUND_att,R_avg_GROUND_landed,R_avg_opp_GROUND_att,R_avg_opp_GROUND_landed,R_avg_CTRL_time(seconds),R_avg_opp_CTRL_time(seconds),R_total_time_fought(seconds),R_total_rounds_fought,R_total_title_bouts,R_current_win_streak,R_current_lose_streak,R_longest_win_streak,R_wins,R_losses,R_draw,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,title_bout,weight_class,B_Stance,R_Stance
10615,0.042269,0.042269,0.510275,0.554311,0.162082,0.309519,0.554895,0.985956,0.121134,0.439433,71.845919,35.988656,103.300618,53.313305,97.817288,59.078458,146.794071,91.288595,4.036597,0.814044,3.687374,1.073329,52.066314,22.143958,72.495353,30.846055,12.413366,8.492503,19.534907,13.202565,7.366239,5.352195,11.270358,9.264685,53.896289,23.582984,71.922628,30.876437,14.045241,9.467657,21.094660,15.142737,3.904390,2.938015,10.283331,7.294132,180.145694,221.912128,755.359089,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.0,166.029558,121.340323,0.000000,0.000000,0.441172,0.419357,0.243634,0.197642,0.269016,0.214391,0.464787,0.189711,78.419699,33.846464,92.823386,37.571582,114.772265,65.622013,113.580555,55.220927,7.735514,1.636909,1.848389,0.647046,65.571751,23.179416,67.895294,18.904783,5.497123,4.648009,8.930164,4.976476,7.350825,6.019039,15.997928,13.690324,53.583723,16.070754,77.918310,28.132993,10.867009,8.553799,9.117870,5.815261,13.968966,9.221911,5.787206,3.623329,317.385101,114.569071,846.117862,19.716398,0.000000,0.000000,0.788656,0.788656,3.154624,3.943280,0.0,0.0,0.788656,2.365968,0.000000,0.000000,0.0,164.170442,163.777698,121.340323,30.577312,31.097903,False,Bantamweight,Orthodox,Southpaw
8366,0.000000,0.000000,0.230874,0.181651,0.000000,0.480582,0.000000,0.038836,0.000000,0.000000,18.310688,4.349523,17.233016,3.310688,18.582539,4.621375,19.

In [ ]:
X_val

,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,B_avg_opp_REV,B_avg_SIG_STR_att,B_avg_SIG_STR_landed,B_avg_opp_SIG_STR_att,B_avg_opp_SIG_STR_landed,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_avg_opp_TOTAL_STR_att,B_avg_opp_TOTAL_STR_landed,B_avg_TD_att,B_avg_TD_landed,B_avg_opp_TD_att,B_avg_opp_TD_landed,B_avg_HEAD_att,B_avg_HEAD_landed,B_avg_opp_HEAD_att,B_avg_opp_HEAD_landed,B_avg_BODY_att,B_avg_BODY_landed,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,B_avg_LEG_att,B_avg_LEG_landed,B_avg_opp_LEG_att,B_avg_opp_LEG_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,B_avg_opp_DISTANCE_att,B_avg_opp_DISTANCE_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_opp_CLINCH_att,B_avg_opp_CLINCH_landed,B_avg_GROUND_att,B_avg_GROUND_landed,B_avg_opp_GROUND_att,B_avg_opp_GROUND_landed,B_avg_CTRL_time(seconds),B_avg_opp_CTRL_time(seconds),B_total_time_fought(seconds),B_total_rounds_fought,B_total_title_bouts,B_current_win_streak,B_current_lose_streak,B_longest_win_streak,B_wins,B_losses,B_draw,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_Height_cms,B_Weight_lbs,R_avg_KD,R_avg_opp_KD,R_avg_SIG_STR_pct,R_avg_opp_SIG_STR_pct,R_avg_TD_pct,R_avg_opp_TD_pct,R_avg_SUB_ATT,R_avg_opp_SUB_ATT,R_avg_REV,R_avg_opp_REV,R_avg_SIG_STR_att,R_avg_SIG_STR_landed,R_avg_opp_SIG_STR_att,R_avg_opp_SIG_STR_landed,R_avg_TOTAL_STR_att,R_avg_TOTAL_STR_landed,R_avg_opp_TOTAL_STR_att,R_avg_opp_TOTAL_STR_landed,R_avg_TD_att,R_avg_TD_landed,R_avg_opp_TD_att,R_avg_opp_TD_landed,R_avg_HEAD_att,R_avg_HEAD_landed,R_avg_opp_HEAD_att,R_avg_opp_HEAD_landed,R_avg_BODY_att,R_avg_BODY_landed,R_avg_opp_BODY_att,R_avg_opp_BODY_landed,R_avg_LEG_att,R_avg_LEG_landed,R_avg_opp_LEG_att,R_avg_opp_LEG_landed,R_avg_DISTANCE_att,R_avg_DISTANCE_landed,R_avg_opp_DISTANCE_att,R_avg_opp_DISTANCE_landed,R_avg_CLINCH_att,R_avg_CLINCH_landed,R_avg_opp_CLINCH_att,R_avg_opp_CLINCH_landed,R_avg_GROUND_att,R_avg_GROUND_landed,R_avg_opp_GROUND_att,R_avg_opp_GROUND_landed,R_avg_CTRL_time(seconds),R_avg_opp_CTRL_time(seconds),R_total_time_fought(seconds),R_total_rounds_fought,R_total_title_bouts,R_current_win_streak,R_current_lose_streak,R_longest_win_streak,R_wins,R_losses,R_draw,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,title_bout,weight_class,B_Stance,R_Stance
9571,0.613178,0.022804,0.468202,0.335091,0.459752,0.194978,0.027079,0.640257,0.410211,0.051308,118.862825,53.853202,78.265309,30.466259,152.313430,82.647616,125.242273,71.821621,2.474473,1.464497,3.619464,1.295480,92.615954,32.493407,67.736242,24.139447,21.229259,16.918432,6.422835,3.376515,5.017613,4.441363,4.106232,2.950297,85.912303,30.336864,72.638869,26.550880,8.925633,7.347958,3.620687,2.310115,24.024889,16.168379,2.005753,1.605264,205.979086,175.949848,740.725725,11.283725,0.0,2.182429,0.000000,2.182429,2.729717,1.364858,0.0,0.0,0.182429,0.547288,1.817571,0.000000,0.0,179.190110,141.385021,0.364858,0.000000,0.439372,0.524375,0.401376,0.514307,2.043927,0.182429,0.638727,0.408785,51.232612,24.285510,26.659340,15.072161,97.245169,67.962344,62.470640,48.598836,4.759302,1.865083,3.802778,2.437920,40.033605,16.385471,20.159115,9.571936,5.134016,3.294932,4.839534,4.171672,6.064990,4.605107,1.660691,1.328553,36.890167,13.359022,15.097260,6.426263,4.357237,2.915731,1.149709,1.021964,9.985208,8.010757,10.412371,7.623935,280.303950,360.142988,863.975786,13.628421,0.000000,1.000000,0.000000,2.635142,3.452712,1.635142,0.0,0.0,0.182429,2.452712,0.000000,0.817571,0.0,169.493480,171.570110,141.385021,30.087854,30.277004,False,Bantamweight,Orthodox,Orthodox
7124,0.154484,0.308969,0.469369,0.518588,0.142126,0.071578,0.051495,0.514948,0.102990,0.154484,61.277361,26.455156,75.554853,38.760832,73.375346,37.677730,85.915317,48.7

### Normalization using MinMax

Scale numeric

In [ ]:
num_scaler = MinMaxScaler()

X_train_num_scaled = num_scaler.fit_transform(X_train.select_dtypes(exclude=["object", "bool"]))

In [ ]:
X_val_num_scaled = num_scaler.transform(X_val.select_dtypes(exclude=["object", "bool"]))

In [ ]:
X_train[ufc_num.columns] = X_train_num_scaled
X_val[ufc_num.columns] = X_val_num_scaled

## Training

We will test out the predictions of these models:

1. Random Forest Classifier
2. Naive Bayes
3. Support Vector Machine
4. CatBoost

### Attempting Random Forest Classifier

Since random forest can't handle categorical values raw, we need to encode them using one-hot encoding first

In [ ]:
cat_enc = OneHotEncoder()

encoded_cats = cat_enc.fit_transform(X_train[ufc_cat.columns])

In [ ]:
X_train_rf = X_train.copy()
X_val_rf = X_val.copy()

In [ ]:
X_train_rf = X_train_rf.drop(axis=1, columns=ufc_cat.columns, errors="ignore")
X_train_rf[cat_enc.get_feature_names_out()] = encoded_cats.toarray()

X_val_rf = X_val.drop(axis=1, columns=ufc_cat.columns, errors="ignore")
X_val_rf[cat_enc.get_feature_names_out()] = cat_enc.transform(X_val[ufc_cat.columns]).toarray()

C:\Users\nayya\AppData\Local\Temp\ipykernel_8296\2621362632.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_val_rf[cat_enc.get_feature_names_out()] = cat_enc.transform(X_val[ufc_cat.columns]).toarray()
C:\Users\nayya\AppData\Local\Temp\ipykernel_8296\2621362632.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_val_rf[cat_enc.get_feature_names_out()] = cat_enc.transform(X_val[ufc_cat.columns]).toarray()
C:\Users\nayya\AppData\Local\Temp\ipykernel_8296\2621362632.py:5: PerformanceWarning: DataFrame is highly fragmented. 

In [ ]:
rf = RandomForestClassifier(random_state=42, n_jobs=-1)
rf.fit(X_train_rf, y_train)

c:\Users\nayya\AppData\Local\pypoetry\Cache\virtualenvs\lab-1-rqMsY7Ti-py3.11\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(n_jobs=-1, random_state=42)

In [ ]:
y_preds_rf = rf.predict(X_val_rf)

In [ ]:
evaluate_classifier_performance(y_preds_rf, y_val["Winner"].to_numpy())

Hasil Evaluasi berdasarkan classification report 

              precision    recall  f1-score   support

        Blue       0.76      0.78      0.77       716
        Draw       1.00      1.00      1.00       716
         Red       0.77      0.76      0.77       717

    accuracy                           0.85      2149
   macro avg       0.85      0.85      0.85      2149
weighted avg       0.84      0.85      0.84      2149



Confusion Matrix



prediction,Blue,Draw,Red
actual,,,
Blue,556,2,158
Draw,0,715,1
Red,172,0,545




Butuh informasi lebih lengkap? silakan simak di bawah ini : 
Accuracy Average: 0.8450442066077245
F1 Macro Average: 0.8450183464142818
F1 Micro Average: 0.8450442066077245
Precision Macro Average: 0.8450315302433502
Precision Micro Average: 0.8450442066077245
Recall Macro Average: 0.8450837469385423
Recall Micro Average: 0.8450442066077245



### Attempting Naive Bayes

A pretty good classifier is Naive Bayes and we'll be using the Complement variant for this due to the imbalanced nature of the dataset (despite being handled using SMOTE-NC and Stratified Sampling)

In [ ]:
X_train_nb = X_train_rf.copy()
X_val_nb = X_val_rf.copy()

In [ ]:
nb = ComplementNB()
nb.fit(X_train_nb, y_train)

c:\Users\nayya\AppData\Local\pypoetry\Cache\virtualenvs\lab-1-rqMsY7Ti-py3.11\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ComplementNB()

In [ ]:
y_preds_nb = nb.predict(X_val_nb)

In [ ]:
evaluate_classifier_performance(y_preds_nb, y_val["Winner"].to_numpy())

Hasil Evaluasi berdasarkan classification report 

              precision    recall  f1-score   support

        Blue       0.46      0.36      0.40       716
        Draw       0.48      0.60      0.53       716
         Red       0.54      0.51      0.52       717

    accuracy                           0.49      2149
   macro avg       0.49      0.49      0.49      2149
weighted avg       0.49      0.49      0.49      2149



Confusion Matrix



prediction,Blue,Draw,Red
actual,,,
Blue,260,268,188
Draw,161,430,125
Red,149,204,364




Butuh informasi lebih lengkap? silakan simak di bawah ini : 
Accuracy Average: 0.4904606793857608
F1 Macro Average: 0.4860377156591768
F1 Micro Average: 0.4904606793857608
Precision Macro Average: 0.4901749762410799
Precision Micro Average: 0.4904606793857608
Recall Macro Average: 0.4904526672016913
Recall Micro Average: 0.4904606793857608



### Attempting Support Vector Machine

Another noteworthy model not used in class is the Support Vector Machine so we'd like to test it out.

In [ ]:
X_train_svc, X_val_svc = X_train_nb.copy(), X_val_nb.copy()

In [ ]:
svc = SVC(random_state=42)
svc.fit(X_train_svc, y_train)

c:\Users\nayya\AppData\Local\pypoetry\Cache\virtualenvs\lab-1-rqMsY7Ti-py3.11\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(random_state=42)

In [ ]:
y_preds_svc = svc.predict(X_val_svc)

In [ ]:
evaluate_classifier_performance(y_preds_svc, y_val["Winner"].to_numpy())

Hasil Evaluasi berdasarkan classification report 

              precision    recall  f1-score   support

        Blue       0.66      0.68      0.67       716
        Draw       0.85      0.97      0.91       716
         Red       0.74      0.62      0.67       717

    accuracy                           0.76      2149
   macro avg       0.75      0.76      0.75      2149
weighted avg       0.75      0.76      0.75      2149



Confusion Matrix



prediction,Blue,Draw,Red
actual,,,
Blue,485,78,153
Draw,17,691,8
Red,229,41,447




Butuh informasi lebih lengkap? silakan simak di bawah ini : 
Accuracy Average: 0.7552349930200093
F1 Macro Average: 0.750235027746251
F1 Micro Average: 0.7552349930200093
Precision Macro Average: 0.7505861601255338
Precision Micro Average: 0.7552349930200093
Recall Macro Average: 0.7552963543005852
Recall Micro Average: 0.7552349930200093



### Attempting CatBoost

A particular model known for being good at classification is CatBoost which combines principles of ensemble learning such as gradient boosting and is good on handling categorical features and targets

In [ ]:
cb = CatBoostClassifier(random_state=42)
cb.fit(X_train, y_train, cat_features=ufc_cat.columns.tolist())

Learning rate set to 0.088301
0:	learn: 1.0716897	total: 216ms	remaining: 3m 36s
1:	learn: 1.0491047	total: 404ms	remaining: 3m 21s
2:	learn: 1.0328687	total: 575ms	remaining: 3m 11s
3:	learn: 1.0140927	total: 783ms	remaining: 3m 15s
4:	learn: 1.0009840	total: 937ms	remaining: 3m 6s
5:	learn: 0.9880883	total: 1.12s	remaining: 3m 5s
6:	learn: 0.9712823	total: 1.45s	remaining: 3m 25s
7:	learn: 0.9547270	total: 1.64s	remaining: 3m 22s
8:	learn: 0.9402818	total: 1.82s	remaining: 3m 20s
9:	learn: 0.9288957	total: 2.11s	remaining: 3m 28s
10:	learn: 0.9179077	total: 2.28s	remaining: 3m 25s
11:	learn: 0.9095211	total: 2.44s	remaining: 3m 21s
12:	learn: 0.8976387	total: 2.63s	remaining: 3m 19s
13:	learn: 0.8875509	total: 2.79s	remaining: 3m 16s
14:	learn: 0.8794517	total: 2.93s	remaining: 3m 12s
15:	learn: 0.8704161	total: 3.21s	remaining: 3m 17s
16:	learn: 0.8586275	total: 3.53s	remaining: 3m 24s
17:	learn: 0.8486899	total: 3.69s	remaining: 3m 21s
18:	learn: 0.8432614	total: 3.9s	remaining: 3m

In [ ]:
y_preds = cb.predict(X_val)

In [ ]:
evaluate_classifier_performance(np.reshape(y_preds, -1), y_val["Winner"].to_numpy())

Hasil Evaluasi berdasarkan classification report 

              precision    recall  f1-score   support

        Blue       0.80      0.75      0.77       716
        Draw       0.99      0.99      0.99       716
         Red       0.76      0.81      0.78       717

    accuracy                           0.85      2149
   macro avg       0.85      0.85      0.85      2149
weighted avg       0.85      0.85      0.85      2149



Confusion Matrix



prediction,Blue,Draw,Red
actual,,,
Blue,535,4,177
Draw,0,711,5
Red,137,2,578




Butuh informasi lebih lengkap? silakan simak di bawah ini : 
Accuracy Average: 0.8487668683108422
F1 Macro Average: 0.8486282457956781
F1 Micro Average: 0.8487668683108422
Precision Macro Average: 0.849429689111202
Precision Micro Average: 0.8487668683108422
Recall Macro Average: 0.8487867147669396
Recall Micro Average: 0.8487668683108422



Since CatBoost is the best one, that's what we'll go with.

## Custom Transformers

For the test set, we created a transformer that will make the code more readable

### Categorical - Remove Identifier or Non-Impactful Features

In [ ]:
class RemoveLessUsefulCategoricalFeatures(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        less_useful_cat_features = ["R_fighter", "B_fighter", "location", "date", "Referee"]
        return X.drop(axis=1, columns=less_useful_cat_features, errors="ignore")

## Test Set on Kaggle

We ran through basically the same steps.

For the categorical columns we:

1. Remove the not so useful features
2. Transforming booleans into strings for CatBoost
3. Impute missing values using most frequent values (mode)
4. One-Hot Encoding (if necessary)

For the numerical features we:

1. Imputed missing values using KNN imputation
2. Scaled using MinMax scaling

In [ ]:
ufc_test = pd.read_csv("./UFC_Test_Classif_X.csv")

In [ ]:
ufc_test_cat = ufc_test.select_dtypes(include=["object", "bool"])
ufc_test_num = ufc_test.drop(axis=1, columns=ufc_test_cat.columns).drop(axis=1, columns=["id"])

In [ ]:
rem_tf = RemoveLessUsefulCategoricalFeatures()
ufc_test_cat = rem_tf.fit_transform(ufc_test_cat)

In [ ]:
ufc_test_cat[ufc_test_cat.select_dtypes(include=["bool"]).columns] = ufc_test_cat.select_dtypes(include=["bool"]).replace({True: "True", False: "False"})

In [ ]:
ufc_test_cat = pd.DataFrame(cat_imputer.transform(ufc_test_cat), columns=ufc_test_cat.columns)

In [ ]:
ufc_test_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602 entries, 0 to 601
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title_bout    602 non-null    object
 1   weight_class  602 non-null    object
 2   B_Stance      602 non-null    object
 3   R_Stance      602 non-null    object
dtypes: object(4)
memory usage: 18.9+ KB


In [ ]:
ufc_test_num = pd.DataFrame(num_imputer.transform(ufc_test_num), columns=ufc_test_num.columns)
ufc_test_num.head()

,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,B_avg_opp_REV,B_avg_SIG_STR_att,B_avg_SIG_STR_landed,B_avg_opp_SIG_STR_att,B_avg_opp_SIG_STR_landed,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_avg_opp_TOTAL_STR_att,B_avg_opp_TOTAL_STR_landed,B_avg_TD_att,B_avg_TD_landed,B_avg_opp_TD_att,B_avg_opp_TD_landed,B_avg_HEAD_att,B_avg_HEAD_landed,B_avg_opp_HEAD_att,B_avg_opp_HEAD_landed,B_avg_BODY_att,B_avg_BODY_landed,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,B_avg_LEG_att,B_avg_LEG_landed,B_avg_opp_LEG_att,B_avg_opp_LEG_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,B_avg_opp_DISTANCE_att,B_avg_opp_DISTANCE_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_opp_CLINCH_att,B_avg_opp_CLINCH_landed,B_avg_GROUND_att,B_avg_GROUND_landed,B_avg_opp_GROUND_att,B_avg_opp_GROUND_landed,B_avg_CTRL_time(seconds),B_avg_opp_CTRL_time(seconds),B_total_time_fought(seconds),B_total_rounds_fought,B_total_title_bouts,B_current_win_streak,B_current_lose_streak,B_longest_win_streak,B_wins,B_losses,B_draw,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_Height_cms,B_Weight_lbs,R_avg_KD,R_avg_opp_KD,R_avg_SIG_STR_pct,R_avg_opp_SIG_STR_pct,R_avg_TD_pct,R_avg_opp_TD_pct,R_avg_SUB_ATT,R_avg_opp_SUB_ATT,R_avg_REV,R_avg_opp_REV,R_avg_SIG_STR_att,R_avg_SIG_STR_landed,R_avg_opp_SIG_STR_att,R_avg_opp_SIG_STR_landed,R_avg_TOTAL_STR_att,R_avg_TOTAL_STR_landed,R_avg_opp_TOTAL_STR_att,R_avg_opp_TOTAL_STR_landed,R_avg_TD_att,R_avg_TD_landed,R_avg_opp_TD_att,R_avg_opp_TD_landed,R_avg_HEAD_att,R_avg_HEAD_landed,R_avg_opp_HEAD_att,R_avg_opp_HEAD_landed,R_avg_BODY_att,R_avg_BODY_landed,R_avg_opp_BODY_att,R_avg_opp_BODY_landed,R_avg_LEG_att,R_avg_LEG_landed,R_avg_opp_LEG_att,R_avg_opp_LEG_landed,R_avg_DISTANCE_att,R_avg_DISTANCE_landed,R_avg_opp_DISTANCE_att,R_avg_opp_DISTANCE_landed,R_avg_CLINCH_att,R_avg_CLINCH_landed,R_avg_opp_CLINCH_att,R_avg_opp_CLINCH_landed,R_avg_GROUND_att,R_avg_GROUND_landed,R_avg_opp_GROUND_att,R_avg_opp_GROUND_landed,R_avg_CTRL_time(seconds),R_avg_opp_CTRL_time(seconds),R_total_time_fought(seconds),R_total_rounds_fought,R_total_title_bouts,R_current_win_streak,R_current_lose_streak,R_longest_win_streak,R_wins,R_losses,R_draw,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,0.0,0.00,0.493750,0.448750,0.475000,0.177500,0.00000,0.125000,0.562500,0.000000,88.937500,42.06250,68.81250,28.437500,131.625000,80.312500,145.4375,98.75000,5.25000,2.50000,2.0625,1.250000,60.3125,19.375000,49.812500,13.500,15.125000,13.0000,18.37500,14.62500,13.5000,9.687500,0.625000,0.312500,58.125000,17.68750,44.187500,9.687500,17.937500,15.062500,15.18750,11.187500,12.875000,9.312500,9.437500,7.562500,367.6250,181.6875,900.00000,15.0,0.0,0.0,1.0,2.0,3.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,165.10,115.0,0.062500,0.000000,0.461250,0.266250,0.322500,0.375000,0.000000,0.000000,0.000000,0.000000,158.625000,73.375000,141.062500,37.625000,200.250000,106.500000,152.125000,45.625000,2.56250,0.687500,2.375000,1.000000,117.062500,40.312500,112.562500,20.562500,19.062500,13.562500,23.562500,13.187500,22.500000,19.500000,4.93750,3.875000,147.812500,66.06250,130.812500,32.625000,7.437500,5.375000,6.812500,2.87500,3.375000,1.937500,3.437500,2.125000,215.000000,94.937500,900.000000,15.0,0.0,3.0,0.0,3.0,4.0,1.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,154.94,152.400,115.0,35.0,27.0
1,0.0,0.25,0.473125,0.371875,0.000000,0.203750,0.00000,0.500000,0.187500,0.000000,54.312500,23.75000,53.87500,20.125000,63.687500,32.250000,109.8750,70.37500,0.00000,0.00000,7.8125,2.375000,37.1250,10.937500,37.750000,11.375,10.437500,7.6875,8.00000,3.62500,6.7500,5.125000,8.125000,5.125000,42.687500,16.06250,47.250000,14.562500,11.500000,7.562500,3.37500,2.312500,0.125000,0.125000,3.250000,3.250000,9.6875

In [ ]:
ufc_test_num = pd.DataFrame(num_scaler.transform(ufc_test_num), columns=ufc_test_num.columns)

In [ ]:
ufc_test_prepped = pd.concat([ufc_test_num, ufc_test_cat], axis=1)
ufc_test_prepped.head()

,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,B_avg_opp_REV,B_avg_SIG_STR_att,B_avg_SIG_STR_landed,B_avg_opp_SIG_STR_att,B_avg_opp_SIG_STR_landed,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_avg_opp_TOTAL_STR_att,B_avg_opp_TOTAL_STR_landed,B_avg_TD_att,B_avg_TD_landed,B_avg_opp_TD_att,B_avg_opp_TD_landed,B_avg_HEAD_att,B_avg_HEAD_landed,B_avg_opp_HEAD_att,B_avg_opp_HEAD_landed,B_avg_BODY_att,B_avg_BODY_landed,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,B_avg_LEG_att,B_avg_LEG_landed,B_avg_opp_LEG_att,B_avg_opp_LEG_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,B_avg_opp_DISTANCE_att,B_avg_opp_DISTANCE_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_opp_CLINCH_att,B_avg_opp_CLINCH_landed,B_avg_GROUND_att,B_avg_GROUND_landed,B_avg_opp_GROUND_att,B_avg_opp_GROUND_landed,B_avg_CTRL_time(seconds),B_avg_opp_CTRL_time(seconds),B_total_time_fought(seconds),B_total_rounds_fought,B_total_title_bouts,B_current_win_streak,B_current_lose_streak,B_longest_win_streak,B_wins,B_losses,B_draw,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_Height_cms,B_Weight_lbs,R_avg_KD,R_avg_opp_KD,R_avg_SIG_STR_pct,R_avg_opp_SIG_STR_pct,R_avg_TD_pct,R_avg_opp_TD_pct,R_avg_SUB_ATT,R_avg_opp_SUB_ATT,R_avg_REV,R_avg_opp_REV,R_avg_SIG_STR_att,R_avg_SIG_STR_landed,R_avg_opp_SIG_STR_att,R_avg_opp_SIG_STR_landed,R_avg_TOTAL_STR_att,R_avg_TOTAL_STR_landed,R_avg_opp_TOTAL_STR_att,R_avg_opp_TOTAL_STR_landed,R_avg_TD_att,R_avg_TD_landed,R_avg_opp_TD_att,R_avg_opp_TD_landed,R_avg_HEAD_att,R_avg_HEAD_landed,R_avg_opp_HEAD_att,R_avg_opp_HEAD_landed,R_avg_BODY_att,R_avg_BODY_landed,R_avg_opp_BODY_att,R_avg_opp_BODY_landed,R_avg_LEG_att,R_avg_LEG_landed,R_avg_opp_LEG_att,R_avg_opp_LEG_landed,R_avg_DISTANCE_att,R_avg_DISTANCE_landed,R_avg_opp_DISTANCE_att,R_avg_opp_DISTANCE_landed,R_avg_CLINCH_att,R_avg_CLINCH_landed,R_avg_opp_CLINCH_att,R_avg_opp_CLINCH_landed,R_avg_GROUND_att,R_avg_GROUND_landed,R_avg_opp_GROUND_att,R_avg_opp_GROUND_landed,R_avg_CTRL_time(seconds),R_avg_opp_CTRL_time(seconds),R_total_time_fought(seconds),R_total_rounds_fought,R_total_title_bouts,R_current_win_streak,R_current_lose_streak,R_longest_win_streak,R_wins,R_losses,R_draw,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,title_bout,weight_class,B_Stance,R_Stance
0,0.0,0.000000,0.493750,0.448750,0.475000,0.177500,0.000000,0.017857,0.187500,0.000000,0.232054,0.254924,0.171602,0.140780,0.313075,0.349185,0.359994,0.425647,0.276316,0.227920,0.107426,0.108696,0.183690,0.158811,0.148694,0.107143,0.213028,0.203125,0.301230,0.291237,0.214864,0.187611,0.009058,0.006010,0.163852,0.111946,0.122403,0.060928,0.222826,0.239087,0.144643,0.133185,0.146307,0.194010,0.066933,0.106514,0.360348,0.217590,0.598125,0.168539,0.000,0.0000,0.2,0.1250,0.130435,0.125,0.0,0.0,0.0,0.3,0.0,0.000000,0.0,0.217391,0.000000,0.015625,0.000000,0.461250,0.266250,0.322500,0.375000,0.000000,0.000000,0.000000,0.000000,0.378290,0.349034,0.310710,0.215000,0.454760,0.429223,0.329989,0.225866,0.085417,0.062500,0.148438,0.090909,0.317946,0.235718,0.281406,0.147932,0.268486,0.251157,0.314167,0.304211,0.314376,0.312743,0.063803,0.069912,0.373162,0.347383,0.297301,0.193047,0.090701,0.088115,0.083079,0.056373,0.023936,0.029640,0.033053,0.031250,0.215701,0.116918,0.598125,0.170455,0.0000,0.1875,0.00,0.1875,0.173913,0.071429,0.0,0.0,0.000000,0.4,0.000000,0.000000,0.0,0.043478,0.000000,0.000000,0.607143,0.285714,False,WomenStrawweight,Orthodox,Orthodox
1,0.0,0.087887,0.473125,0.371875,0.000000,0.203750,0.000000,0.071429,0.062500,0.000000,0.141711,0.143939,0.134352,0.099629,0.151483,0.140217,0.271968,0.303341,0.000000,0.000000,0.406916,0.206522,0.113069,0.089652,0.112687,0.090278,0.147007,0.120117,0.131148,0.072187,0.107432,0

In [ ]:
y_test_preds = cb.predict(ufc_test_prepped)

In [ ]:
submissions = ufc_test[["id"]]
submissions["Winner"] = y_test_preds.reshape(-1)
submissions.to_csv("ufc_classification.csv", index=False)

C:\Users\nayya\AppData\Local\Temp\ipykernel_8296\1217007067.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submissions["Winner"] = y_test_preds.reshape(-1)
